# Unit 3: Detect and localise a person

<img src="img/jackal_clearpath_instruction_manual_unit3_intro.png"/>

<img src="img/Clearpath-Logo.png" width="300"/>

<img src="img/robotignite_logo_text.png" width="300"/>

In this unit you are goint to learn how to detect a person using two different systems:<br>
* **Detect through leg detection with laser sensor**: This system detect persons based on laser readings of leg patterns. This allows you to have even the position of the person, not only detect if there is a person or not.
* **Detect person through an RGB camera from the stereo-cam**: This system doesnt give you directly the position of the person, but detects the persons through Video with OpenCV.
* As extra, you are going to learn how to **Get Point cloud** from stereocamera: Use the stereocamera to extract pointcloud data from the scene.

## Person leg Detector

This is a classical way to detect people. Its fast and relatively robust. You will use a ROS package called **leg_detector**, for more information http://wiki.ros.org/leg_detector.<br>

The first step is to generate a launch file that sets all the parameters:

**start_legdetector.launch**

In [ ]:
<launch> 
    <arg name="scan" default="/front/scan" />
    <arg name="machine" default="localhost" />
    <arg name="user" default="" />
    <!-- Leg Detector -->
    <node pkg="leg_detector" type="leg_detector" name="leg_detector" args="scan:=$(arg scan) $(find leg_detector)/config/trained_leg_detector.yaml" respawn="true" output="screen">
        <param name="fixed_frame" type="string" value="map" />
    </node>
    
</launch> 

As you can see here the only thing that you have to set really is the scan topic, which is the topic for the laser. The rest of parameters are set in a yaml file trained_leg_detector.yaml, which you can leave as it is unless you want to optimise or make set all teh peremeters for finer tuning.

In [ ]:
<param name="fixed_frame" type="string" value="map" />

Note this parameter **fixed_frame**. It uses the **map** frame as refference. This means that you **HAVE** to publish the **map** frame one way or another. So before anything else, you have to launch your **navigation** system. You can choose between two options:

* You launch the **start_navigation_with_map.launch**. You can load it with the **mymap_empty.yaml** map. 

In [ ]:
<launch>

    <!-- Run the map server -->
    <arg name="map_file" default="$(find jackal_tools)/maps/mymap.yaml"/>
    <node name="map_server" pkg="map_server" type="map_server" args="$(arg map_file)" />
    
    <!--- Run AMCL -->
    <include file="$(find my_jackal_tools)/launch/amcl.launch" />
    
    <!--- Run Move Base -->
    <include file="$(find my_jackal_tools)/launch/with_map_move_base.launch" />

</launch>

* You launch the **start_navigation_with_gps_ekf.launch** which uses the GPS and so on for localization.

In [ ]:
<launch>
    <!-- Run navsat gps to odometry conversion-->
  <include file="$(find jackal_tools)/launch/start_navsat.launch" />

    
    <!-- Run the ekf for map to odom config -->
    <node pkg="robot_localization" type="ekf_localization_node" name="ekf_localization_with_gps"> 
    <rosparam command="load" file="$(find my_jackal_tools)/config/robot_localization_with_gps.yaml" />
  </node>


    <!-- Run the map server -->
    <arg name="map_file" default="$(find my_jackal_tools)/maps/mymap_empty.yaml"/>
    <node name="map_server" pkg="map_server" type="map_server" args="$(arg map_file)" />
    
    <!--- Run Move Base -->
    <include file="$(find my_jackal_tools)/launch/with_map_move_base.launch" />
 

    <!-- Start RVIZ for Localization -->
    <include file="$(find my_jackal_tools)/launch/view_robot.launch">
        <arg name="config" value="person" />
    </include>

</launch>

This unit will be done by using the **start_navigation_with_gps_ekf.launch**, but you could do all its done here in the same way with the map navigation. Note that you use an rviz file called **person**. You will have to create it for this unit, so that you only have the essentials.

Once you have it, you should be able to launch **start_legdetector.launch** and start detecting people right away. The detections will be published in the topic **/leg_tracker_measurements**. Here you have output example:

In [ ]:
---                                                                                                                                                                       
header:                                                                                                                                                                   
  seq: 668                                                                                                                                                                
  stamp:                                                                                                                                                                  
    secs: 40                                                                                                                                                              
    nsecs: 788000000                                                                                                                                                      
  frame_id: ''                                                                                                                                                            
people:                                                                                                                                                                   
  -                                                                                                                                                                       
    header:                                                                                                                                                               
      seq: 0                                                                                                                                                              
      stamp:                                                                                                                                                              
        secs: 40                                                                                                                                                          
        nsecs: 765000000                                                                                                                                                  
      frame_id: map                                                                                                                                                       
    name: leg_detector                                                                                                                                                    
    object_id: legtrack0                                                                                                                                                  
    pos:                                                                                                                                                                  
      x: 2.95611178591                                                                                                                                                    
      y: -0.127522822998                                                                                                                                                  
      z: 0.334633714587                                                                                                                                                   
    reliability: 0.923895539057                                                                                                                                           
    covariance: [0.10543790524172501, 0.0, 0.0, 0.0, 0.10543790524172501, 0.0, 0.0, 0.0, 10000.0]                                                                         
    initialization: 0                                                                                                                                                     
  -                                                                                                                                                                       
    header:                                                                                                                                                               
      seq: 0                                                                                                                                                              
      stamp:                                                                                                                                                              
        secs: 40                                                                                                                                                          
        nsecs: 765000000                                                                                                                                                  
      frame_id: map                                                                                                                                                       
    name: leg_detector                                                                                                                                                    
    object_id: legtrack1                                                                                                                                                  
    pos:                                                                                                                                                                  
      x: 2.96855903689                                                                                                                                                    
      y: 0.133168364643                                                                                                                                                   
      z: 0.334657142144                                                                                                                                                   
    reliability: 0.939497802961                                                                                                                                           
    covariance: [0.10196496436200578, 0.0, 0.0, 0.0, 0.10196496436200578, 0.0, 0.0, 0.0, 10000.0]                                                                         
    initialization: 0                                                                                                                                                     
cooccurrence: []

As you can see its posible that its making multiple detections, thats why post processing as improvement.

This topic is of type **PositionMeasurementArray**.

In [ ]:
### PositionMeasurementArray

std_msgs/Header header                                                                                                         
  uint32 seq                                                                                                                   
  time stamp                                                                                                                   
  string frame_id                                                                                                              
people_msgs/PositionMeasurement[] people                                                                                       
  std_msgs/Header header                                                                                                       
    uint32 seq                                                                                                                 
    time stamp                                                                                                                 
    string frame_id                                                                                                            
  string name                                                                                                                  
  string object_id                                                                                                             
  geometry_msgs/Point pos                                                                                                      
    float64 x                                                                                                                  
    float64 y                                                                                                                  
    float64 z                                                                                                                  
  float64 reliability                                                                                                          
  float64[9] covariance                                                                                                        
  byte initialization                                                                                                          
float32[] cooccurrence

Here you have Position information and also reliability of the detections.

But lets go a step further. You need to visualise those detections in space. For that by default the leg_detector already has a topic that publishes RVIZ markers called **/visualization_marker**. This is depicts the person detection through a sphere based on the reliability changing its colour and appearance.

<img src="img/jackal_clearpath_instruction_manual_unit3_markerreal2.png"/>

Open RVIZ and add a **Marker** element, and select the topic **/visualization_marker**.

<img src="img/jackal_clearpath_instruction_manual_unit3_markerbasic.png" width="600"/>

You can aslo generate a custom marker that will be much more descriptive when visualised. For that you will have to create a python script that reads the leg_detections from **/leg_tracker_measurements**, and publish it in a new marker_topic with its own custom marker, in this case will be a mesh of the person detected.

**leg_detector_marker_publisher.py**

In [ ]:
#!/usr/bin/env python
import rospy
from people_msgs.msg import PositionMeasurementArray


#!/usr/bin/env python

import rospy
from visualization_msgs.msg import Marker
from geometry_msgs.msg import Point


class MarkerBasics(object):

    def __init__(self):
        self.marker_objectlisher = rospy.Publisher('/marker_person_detected_leg', Marker, queue_size=1)
        self.rate = rospy.Rate(1)
        self.init_marker(index=0,z_val=0)
    
    def init_marker(self,index=0, z_val=0):
        self.marker_object = Marker()
        self.marker_object.header.frame_id = "/map"
        self.marker_object.header.stamp    = rospy.get_rostime()
        self.marker_object.ns = "jackal"
        self.marker_object.id = index
        self.marker_object.type = Marker.MESH_RESOURCE
        self.marker_object.action = Marker.ADD
        
        my_point = Point()
        self.marker_object.pose.position = my_point
        
        self.marker_object.pose.orientation.x = 0
        self.marker_object.pose.orientation.y = 0
        self.marker_object.pose.orientation.z = 1.0
        self.marker_object.pose.orientation.w = 1.0
        self.marker_object.scale.x = 1.0
        self.marker_object.scale.y = 1.0
        self.marker_object.scale.z = 1.0
    
        self.marker_object.color.r = 0.0
        self.marker_object.color.g = 0.0
        self.marker_object.color.b = 0.0
        # This has to be otherwise it will be transparent
        self.marker_object.color.a = 0.0
        self.marker_object.mesh_resource = "package://person_sim/models/person_standing/meshes/standingv2.dae"
        self.marker_object.mesh_use_embedded_materials = True
        # If we want it for ever, 0, otherwise seconds before desapearing
        self.marker_object.lifetime = rospy.Duration(0)

    def update_position(self,position, reliability):
        
        self.marker_object.pose.position = position
        
        self.marker_objectlisher.publish(self.marker_object)

"""
### PositionMeasurementArray

std_msgs/Header header                                                                                                         
  uint32 seq                                                                                                                   
  time stamp                                                                                                                   
  string frame_id                                                                                                              
people_msgs/PositionMeasurement[] people                                                                                       
  std_msgs/Header header                                                                                                       
    uint32 seq                                                                                                                 
    time stamp                                                                                                                 
    string frame_id                                                                                                            
  string name                                                                                                                  
  string object_id                                                                                                             
  geometry_msgs/Point pos                                                                                                      
    float64 x                                                                                                                  
    float64 y                                                                                                                  
    float64 z                                                                                                                  
  float64 reliability                                                                                                          
  float64[9] covariance                                                                                                        
  byte initialization                                                                                                          
float32[] cooccurrence
"""
class LegDetector(object):
    def __init__(self):
        rospy.Subscriber("/leg_tracker_measurements", PositionMeasurementArray, self.leg_detect_callback)

        self.markerbasics_object = MarkerBasics()

    def leg_detect_callback(self,data):
        for people_object in data.people:
            self.markerbasics_object.update_position(people_object.pos, people_object.reliability)
    
    def start_loop(self):
        # spin() simply keeps python from exiting until this node is stopped
        rospy.spin()

if __name__ == '__main__':
    rospy.init_node('leg_detections_listener_node', anonymous=True)
    legdetector_object = LegDetector()
    legdetector_object.start_loop()
    

If you need more information about how this is done, we highly reccomend you to do the RVIZ Markers course in RobotIgniteAcademy.<br>
Here you basically retrieve the position of the leg detection and publish the marker using the mesh **standingv2.dae** as marker.<br>
Bare in mind that because the leg detector doesnt give you orientation information the marker will have always the same orientation unless you detect the orientation of the person by other menas, such as movement direction.

If you execute the python script **leg_detector_marker_publisher.py**, You should get something similar to this by launching instead of your rviz config the one given by RobotIgniteSystem installed packages:

In [ ]:
<!-- Start RVIZ for Localization -->
    <include file="$(find jackal_tools)/launch/view_robot.launch">
        <arg name="config" value="person" />
    </include>

<img src="img/jackal_clearpath_instruction_manual_unit3_rvizmarkerview.png" width="600"/>

As you can see, the elements of the **global** map and **localcostmap** are activated.<br>
Its also selected a diferent camera view, the XYOrbit, that allows you to see better the Person 3D model.

<p style="background:#EE9023;color:white;">Exercise U3-1</p>

Based on the leg detection data, get the direction in which the person is moving and change the orientation of the custom marker to better match the real person.

<p style="background:#EE9023;color:white;">Exercise U3-2</p>

## Pedestrian Detector

Now you are going to use the Stereo RGB camera to detect persons.

### Detect pedestrians with OpenCV script

You are going to learn how to acces the camera data in ROS, and convert it to OpenCV format. Then you will process the image data to detect pedestrians and draw a bounding box around them.

For this you will create two new scripts:<br>
* **the main detect person**, shich will retrieve the image data from the Summit XL robot ROS topics and send it to a pedestrian detector script. 

* **The pedestrian detector script**: This script will do the tough work of detecting and drawing the bounding box around the person detected.

There are **two** main topics for the images, one for each of the two cameras:
* /front/left/image_raw
* /front/right/image_raw

**detect_person_camera.py**

In [ ]:
#!/usr/bin/env python

import rospy
import cv2
import numpy as np
from cv_bridge import CvBridge, CvBridgeError
from geometry_msgs.msg import Twist
from sensor_msgs.msg import Image
from pedestrian_detector import PedestrianDetector

class DetectPersonCam(object):

    def __init__(self):
    
        self.bridge_object = CvBridge()
        self.image_sub = rospy.Subscriber("/front/left/image_raw",Image,self.camera_callback)
        self.pedestrian_detector = PedestrianDetector()
        self.init_time = rospy.get_time()
        self.now_pitch = 0
        self.now_roll = 0
        self.process_this_frame = True
        
    def camera_callback(self,data):
        
        try:
            # We select bgr8 because its the OpneCV encoding by default
            cv_image = self.bridge_object.imgmsg_to_cv2(data, desired_encoding="bgr8")
        except CvBridgeError as e:
            print(e)

        # To process only half of the images for faster performance
        if self.process_this_frame:
            self.pedestrian_detector.detect_pedestrian(cv_image)
        
        self.process_this_frame = not self.process_this_frame
    
    


def main():
    rospy.init_node('detect_person_camera_node', anonymous=True)
    
    detect_personcam_object = DetectPersonCam()
    
    try:
        rospy.spin()
    except KeyboardInterrupt:
        print("Shutting down")
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()

Comment on the following:

In [ ]:
self.image_sub = rospy.Subscriber("/front/left/image_raw",Image,self.camera_callback)

Here you susbrcibe to the topic where the Camera images are published for the left camera. We could use the **right** camera, just pick one.

In [ ]:
cv_image = self.bridge_object.imgmsg_to_cv2(data, desired_encoding="bgr8")

Here you get that image and tranform it into OpenCV BGR8bit format. If you want to know more details, please reffer to the ROS perception Course in RobotIgniteAcademy.

In [ ]:
if self.process_this_frame:
    ...
self.process_this_frame = not self.process_this_frame

This condition makes the program procss only half of the images, making it much faster.

In [ ]:
self.pedestrian_detector.detect_pedestrian(cv_image)

This function sends the data to the pedestrian detector.

**pedestrian_detector.py**

In [ ]:
#! /usr/bin/env python
"""
USAGE
python detect.py --images images
"""
# import the necessary packages
from __future__ import print_function
from imutils.object_detection import non_max_suppression
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2


class PedestrianDetector(object):
    def __init__(self):
        # initialize the HOG descriptor/person detector
        self.hog = cv2.HOGDescriptor()
        self.hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

    def detect_pedestrian(self, image):

    	image = imutils.resize(image, width=min(400, image.shape[1]))
    	orig = image.copy()
    
    	# detect people in the image
    	(rects, weights) = self.hog.detectMultiScale(image, winStride=(4, 4),
    		padding=(8, 8), scale=1.05)
    
    	# draw the original bounding boxes
    	"""
    	for (x, y, w, h) in rects:
    		cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 0, 255), 2)
        """
    	# apply non-maxima suppression to the bounding boxes using a
    	# fairly large overlap threshold to try to maintain overlapping
    	# boxes that are still people
    	rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
    	pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)
    
    	# draw the final bounding boxes
    	for (xA, yA, xB, yB) in pick:
    		cv2.rectangle(image, (xA, yA), (xB, yB), (0, 255, 0), 2)
    
    	# show some information on the number of bounding boxes
    	filename = "LiveCam"
    	print("[INFO] {}: {} original boxes, {} after suppression".format(
    		filename, len(rects), len(pick)))
    
    	# show the output images
    	#cv2.imshow("Before NMS", orig)
    	cv2.imshow("Pedestrian CAM", image)
    	cv2.waitKey(1)

When executing this code you should get something similar to this:

<img src="img/jackal_clearpath_instruction_manual_unit3_persondetectgui2.png"/>

<img src="img/jackal_clearpath_instruction_manual_unit3_persondetectgui3.gif"/>

<p style="background:#EE9023;color:white;">Exercise U3-2</p>

Create a method that based on the data of the pedestrian detection bounding boxes, publishes the positon relative to Jackal robot. For that you will have to make calibration tests to see the size of the bounding box based on the distance, and TF publications. If you need more information on how to do that, in the course ** ROS TF** and **ROS Perception** you will it.

<p style="background:#EE9023;color:white;">END Exercise U3-2</p>

## Generate PointCloud from stereo camera

So having a stereo camera it alaows you to use the ROS package <a href="http://wiki.ros.org/stereo_image_proc">stereo_image_proc</a>.<br>
This package converts two RGB images into a Pointcloud, allowing you to detect objects in space with only two cameras.

To launch it you will need to create two launches:

**start_stereovision.launch**

In [ ]:
<launch>
 <!-- -->
  <node pkg="stereo_image_proc" type="stereo_image_proc" name="" respawn="true">
    <remap from="left/image_raw" to="/front/left/image_raw" />
    <remap from="left/camera_info" to="/front/left/camera_info" />
    <remap from="right/image_raw" to="/front/right/image_raw" />
    <remap from="right/camera_info" to="/front/right/camera_info" />
  </node>

</launch>

Here you just have to remap the left and right image topic for the ones of your robot, in this case Jackals **left/image_raw** and **right/image_raw**. Its also important the camera info topics.

Once you have it you are good to go. This among other topics will piblish two very usefull topics:
* **/points2**: This contains the point cloud generated.
* **/disparity**: This contains the disparity image, which is an image that give you an idea of the 3D vision in a 2D image. Very usefull to debug stereocam errors in detections.

To see the **/disparity** topic , becsue it has a spacial type **stereo_msgs/DisparityImage**, it cant be visualised by normal image visualization. So you need to launch a node spacialy for that.

**start_stereovision_disparityimage.launch**

In [ ]:
<launch>
 <!-- -->
  <node pkg="image_view" type="stereo_view" name="" respawn="true" output="screen">
    <!-- Remp to slash to make it empty, otherwise ros doesnt allow empty remappings, but we need it to avoid its value is stereo"-->
    <remap from="stereo" to="/"/>
    <remap from="image" to="image_rect"/>
  </node>

</launch>

Here you state the **namespace** of the rectified images published. If you didnt remap them ( which is the nomal way), it has to be empty. This is because the rectified images by the **stereo_image_proc** node are just **/right/image_rect** for example. No namespace. So you have to just put the value in stereo = **/**. And the **image** the **image_rect**.

So you are ready, launch both **start_stereovision.launch** and **start_stereovision_disparityimage.launch**, and you will have something similar to this in the Graphical Tools window:

<img src="img/jackal_clearpath_instruction_manual_unit3_stereovision3.png"/>

And if you add a **PointCloud Element** in RVIZ and set the **/point2** topic, you should get something similar to this:

<img src="img/jackal_clearpath_instruction_manual_unit3_stereovision2.png"/>

<p style="background:#EE9023;color:white;">EXTRA Exercise U3-3</p>

Create a script that with the pointcloud data publishes the TF of camera_link to **person_detected**.
Once you have it working, then create a main script that makes Jackal move to the person detected. 

<p style="background:#EE9023;color:white;">EXTRA END Exercise U3-3</p>

## Congratulations, you have finished the theory of this course, now to the micro project!